# BigQuery_Table_Data_Source

### Source Data
##### Overview

This notebook imports source data for this project into Google BigQuery. All the remaining notebooks utilize BigQuery as the source and leverage API's native to the machine learning approaches they feature.

In the enviornment setup notebook (00), a BigQuery source table was exported to CSV format in a Cloud Storage Bucket. This notebook, 01 - BigQuery - Table Data Source, starts the machine learning lifecycle by importing source data and preparing it for machine learning.

All of these workflows utilize tabular data to fit a supervised learning model: predict a target variable by learning patterns in feature columns. The type of supervised learning used in these projects is classification: models with a target variable that has multiple discrete classes.

##### The Data

The source data is first exported to Google Cloud Storage in CSV format below. The BigQuery source table is bigquery-public-data.ml_datasets.ulb_fraud_detection. This is a table of credit card transactions that are classified as fradulant, Class = 1, or normal Class = 0.

- The data can be researched further at this Kaggle link.
- Read mode about BigQuery public datasets here

##### Description of the Data

This is a table of 284,807 credit card transactions classified as fradulant or normal in the column Class. In order protect confidentiality, the original features have been transformed using principle component analysis (PCA) into 28 features named V1, V2, ... V28 (float). Two descriptive features are provided without transformation by PCA:

- Time (integer) is the seconds elapsed between the transaction and the earliest transaction in the table
- Amount (float) is the value of the transaction

##### Quick Note on PCA

- PCA is an unsupervised learning technique: there is not a target variable. PCA is commonlly used as a variable/feature reduction technique. If you have 100 features then you could reduce it to a number p (say 10) projected features. The choice of this number is a balance of how well it can explain the variance of the full feature space and reducing the number of features. Each projected feature is orthogonal to each other feature, meaning there is no correlation between these new projected features.

##### Preparation of the Data

This notebook adds two columns to the source data and stores it in a new table with suffix _prepped.

- transaction_id (string) a unique id for the row/transaction
- splits (string) this divided the tranactions into sets for TRAIN (80%), VALIDATE (10%), and TEST (10%)

### Setup
inputs:

In [1]:
REGION = "us-central1"
PROJECT_ID = "mrnaif-demo"
DATANAME = "fraud"
NOTEBOOK = '01'

In [2]:
from google.cloud import bigquery

In [3]:
bq = bigquery.Client(project = PROJECT_ID)

In [4]:
BUCKET = PROJECT_ID

### Create Dataset
List BigQuery datasets in the project:

In [6]:
query = f"""
SELECT schema_name
FROM `{PROJECT_ID}.INFORMATION_SCHEMA.SCHEMATA`
"""
bq.query(query = query).to_dataframe()

,schema_name


Create the dataset if missing:

In [7]:
query = f"""
CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.{DATANAME}`
OPTIONS(
     location = '{REGION}',
     labels = [('notebook','{NOTEBOOK}')]
)
"""
job = bq.query(query = query)
job.result()

In [10]:
(job.ended-job.started).total_seconds()

0.699

### Create Table
- import data from CLoud Storage Bucket
- https://cloud.google.com/bigquery/docs/loading-data-cloud-storage-csv

In [13]:
destination = bigquery.TableReference.from_string(f"{PROJECT_ID}.{DATANAME}.{DATANAME}")
job_config = bigquery.LoadJobConfig(
    write_disposition = 'WRITE_TRUNCATE',
    source_format = bigquery.SourceFormat.CSV,
    autodetect = True,
    labels = {'notebook':f'{NOTEBOOK}'}
)
job = bq.load_table_from_uri(f"gs://{BUCKET}/{DATANAME}/data/{DATANAME}.csv", destination, job_config = job_config)
job.result()

LoadJob<project=mrnaif-demo, location=us-central1, id=7287d42c-aa00-461d-ae47-8f4f24556109>

In [15]:
(job.ended-job.started).total_seconds()

10.315

In [16]:
query = f"""
SELECT *
FROM `{DATANAME}.{DATANAME}`
LIMIT 5
"""
bq.query(query = query).to_dataframe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,150336,2.030851,-0.235559,-2.804940,-0.547376,2.441730,3.277629,-0.652959,0.843118,0.542395,...,-0.058377,-0.092950,0.203618,0.642094,-0.105975,0.502090,-0.024706,-0.040995,0.78,0
1,84068,-0.679466,1.334849,1.367987,0.856085,0.000189,-0.701735,0.582134,-0.003885,-0.741823,...,0.077499,0.311384,-0.095417,0.413387,-0.212580,-0.457638,-0.250283,-0.139162,0.78,0
2,122091,2.141596,0.077735,-2.104923,0.061232,0.368903,-1.825884,0.748172,-0.564815,0.354072,...,0.131269,0.490071,-0.035338,0.008649,0.415350,0.245709,-0.091809,-0.081979,0.78,0
3,136167,2.073311,0.265580,-1.745544,0.492756,0.286109,-1.427709,0.310907,-0.422277,0.505541,...,0.191552,0.767662,-0.043917,-0.156863,0.295408,-0.096150,0.002101,-0.031715,0.78,0
4,150523,2.032967,-0.387542,-0.505876,0.350922,-0.424833,-0.134032,-0.623411,-0.005357,1.481508,...,0.151034,0.737664,0.044700,-0.682962,-0.007459,-0.166488,0.047286,-0.044997,0.78,0


### Check out this table in BigQuery Console:
- Click: https://console.cloud.google.com/bigquery
- Make sure project selected is the one from this notebook
- Under Explore, expand this project and review the dataset and table

### Review Data
- Additional SQL queries could be used to review the data. This section shows moving the table to a Pandas dataframe for local review in Python:

In [17]:
query = f"""
SELECT *
FROM `{DATANAME}.{DATANAME}`
"""
df = bq.query(query = query).to_dataframe()

In [18]:
df['Class'].value_counts()

Class
0    284315
1       492
Name: count, dtype: Int64

In [20]:
df['Class'].value_counts(normalize=True)

Class
0    0.998273
1    0.001727
Name: proportion, dtype: Float64

Highly Imbalanced!

In [19]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

### Prepare Data for Analysis
Create a prepped version of the data with train/test split using SQL DDL:

In [23]:
query = f"""
CREATE OR REPLACE TABLE `{DATANAME}.{DATANAME}_prepped` AS
WITH add_id AS(SELECT *, GENERATE_UUID() transaction_id FROM `{DATANAME}.{DATANAME}`)
SELECT*,
    CASE
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 8 THEN "TRAIN"
        WHEN MOD(ABS(FARM_FINGERPRINT(transaction_id)),10) < 9 THEN "VALIDATE"
        ELSE "TEST"
    END AS splits
FROM add_id
"""
job = bq.query(query = query)
job.result()

In [24]:
(job.ended-job.started).total_seconds()

7.391

In [25]:
job.estimated_bytes_processed/1000000 #MB

70.632136

Review the train/test split:

In [26]:
query = f"""
SELECT splits, count(*) as Count, 100*count(*) / (sum(count(*)) OVER()) as Percentage
FROM `{DATANAME}.{DATANAME}_prepped`
GROUP BY splits
"""
bq.query(query=query).to_dataframe()

,splits,Count,Percentage
0,TEST,28662,10.063657
1,VALIDATE,28217,9.907411
2,TRAIN,227928,80.028932


In [27]:
query = f"""
SELECT *
FROM `{DATANAME}.{DATANAME}_prepped`
LIMIT 5
"""
data = bq.query(query = query).to_dataframe()

In [28]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,10583,-1.202598,0.635702,3.377880,2.234869,0.237657,0.424087,0.068234,0.035178,0.522148,...,0.061271,0.471488,0.193197,-0.108628,-0.205586,-0.170543,0.0,0,e4a2071d-51ba-4e85-9e03-2573f457a138,TEST
1,41142,1.172413,0.659021,-0.022943,2.238594,0.618110,0.143063,0.377312,0.022806,-1.302248,...,-0.125425,-0.295692,0.673693,0.100964,-0.026671,-0.007202,0.0,0,b0c83024-5e8c-4fb4-b71f-72c08fc26fd9,TEST
2,94817,-0.311869,1.113026,2.050264,3.458307,0.749853,1.357551,0.135304,0.148008,0.078646,...,-0.045211,0.618446,-0.772424,0.046515,-0.097679,-0.037401,0.0,0,17cc17a4-ead1-475b-9c61-f85e12b0ee4a,TEST
3,114084,-2.956093,-3.336607,0.599052,4.228798,4.343250,-2.446534,-2.009516,0.205328,-0.226058,...,0.549155,0.084735,-1.660483,-0.238725,0.084017,0.960471,0.0,0,1aaa8454-721f-4e21-8ef0-015d6d15272f,TEST
4,140315,1.882751,0.532175,0.150708,3.889469,0.011037,0.101617,-0.105079,-0.027678,-0.649422,...,0.487635,-0.024250,-0.512936,-0.516050,0.011179,-0.026608,0.0,0,8b43188e-da87-4e6c-b76d-a321a84c6b52,TEST
